In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("yarn") \
    .appName("pantheon") \
    .getOrCreate()

In [2]:
stazioneDf = spark.read.format('com.databricks.spark.csv')\
.options(delimiter=';',header='true', inferschema='true').load('input/pantheon20190612-stazione.csv')
stazioneDf.take(1)

[Row(id_dato=1811253, data_ora=datetime.datetime(2018, 10, 12, 14, 25, 15), temp1_media=25.99, temp1_min=25.93, temp1_max=26.08, ur1_media=56.69, ur1_min=52.56, ur1_max=58.97, temp1_ur1_n_letture=5, pioggia_mm=0.0, rad W/mq=75.0, rad_n_letture=5, wind_dir=6, wind_dir_n_letture=5, wind_speed_media=0.3, wind_speed_max=15.62, wind_speed_n_letture=25, pressione_mbar=1085.94, pressione_standard_mbar=1118.68, pressione_n_letture=5, rad W/mq array='{}')]

In [3]:
stazioneDf.printSchema()

root
 |-- id_dato: integer (nullable = true)
 |-- data_ora: timestamp (nullable = true)
 |-- temp1_media: double (nullable = true)
 |-- temp1_min: double (nullable = true)
 |-- temp1_max: double (nullable = true)
 |-- ur1_media: double (nullable = true)
 |-- ur1_min: double (nullable = true)
 |-- ur1_max: double (nullable = true)
 |-- temp1_ur1_n_letture: integer (nullable = true)
 |-- pioggia_mm: double (nullable = true)
 |-- rad W/mq: double (nullable = true)
 |-- rad_n_letture: integer (nullable = true)
 |-- wind_dir: integer (nullable = true)
 |-- wind_dir_n_letture: integer (nullable = true)
 |-- wind_speed_media: double (nullable = true)
 |-- wind_speed_max: double (nullable = true)
 |-- wind_speed_n_letture: integer (nullable = true)
 |-- pressione_mbar: double (nullable = true)
 |-- pressione_standard_mbar: double (nullable = true)
 |-- pressione_n_letture: integer (nullable = true)
 |-- rad W/mq array: string (nullable = true)



In [11]:
#questa cella serve ad individuare i blocchi del dataframe accomunati da una caratteristica comune, ad esempio in questo caso
# vogliamo raggruppare tutti i blocchi con radiazione nulla e tutti quelli con radiazione non nulla
# la funzione lag serve , data una certa riga , a selezionare la riga che si trova ad un certo offset da questa, nel nostro caso 1
# usiamo lag per selezionare la riga subito sotto alla riga di inizio di un blocco 
# coalesce serve per selezionare, data una lista di colonne, la prima colonna non nulla
# count considera nel conteggio solo le celle non vuote, così può essere utile per individuare una sottocolonna con tutti valori non nulli
import pyspark.sql.functions as F
from pyspark.sql.window import Window
nullRadSlot = stazioneDf
nullRadSlot = nullRadSlot.withColumn("isnull", F.when(nullRadSlot['rad W/mq'].isNull(), True).otherwise(False))
nullRadSlot = nullRadSlot.withColumn("lag_isnull", F.lag(nullRadSlot["isnull"],1).over(Window.orderBy(nullRadSlot["data_ora"])))
nullRadSlot = nullRadSlot.withColumn("change", F.coalesce(nullRadSlot["isnull"]!=nullRadSlot["lag_isnull"],F.lit(False)))
nullRadSlot = nullRadSlot.withColumn("block", F.sum(nullRadSlot["change"].cast("int")).over(Window.orderBy(nullRadSlot["data_ora"])))\
  .groupBy("block")\
  .agg(F.min(nullRadSlot["data_ora"]).alias('mindata'),
    F.max(nullRadSlot["data_ora"]).alias('maxdata'),
    (F.count(nullRadSlot['rad W/mq'])==0).alias('blocco_isnull'))

In [98]:
nullRadSlot.show()

+-----+-------------------+-------------------+-------------+
|block|            mindata|            maxdata|blocco_isnull|
+-----+-------------------+-------------------+-------------+
|    0|2018-10-12 14:25:15|2019-05-06 08:55:28|        false|
|    1|2019-05-06 13:40:10|2019-06-12 09:25:15|         true|
+-----+-------------------+-------------------+-------------+



In [101]:
timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"
timeDiff = (F.unix_timestamp('maxdata', format=timeFmt)
            - F.unix_timestamp('mindata', format=timeFmt))
nullRadSlot = nullRadSlot.withColumn("Duration", timeDiff)

In [102]:
nullRadSlot.show()

+-----+-------------------+-------------------+-------------+--------+
|block|            mindata|            maxdata|blocco_isnull|Duration|
+-----+-------------------+-------------------+-------------+--------+
|    0|2018-10-12 14:25:15|2019-05-06 08:55:28|        false|17778613|
|    1|2019-05-06 13:40:10|2019-06-12 09:25:15|         true| 3181505|
+-----+-------------------+-------------------+-------------+--------+



# Verifica dei valori nulli

In [3]:
stazioneDf.filter(stazioneDf.id_dato.isNotNull()).count()

30802

In [4]:
stazioneDf.filter(stazioneDf.temp1_media.isNotNull()).count()

30802

In [5]:
stazioneDf.filter(stazioneDf.ur1_media.isNotNull()).count()

30802

In [6]:
stazioneDf.filter(stazioneDf.wind_speed_media.isNotNull()).count()

22525

In [7]:
stazioneDf.filter(stazioneDf['rad W/mq'].isNotNull()).count()

20206

## Calcolo valori medi

In [12]:
def fill_with_mean(df, include=set()): 
    stats = df.agg(*(
        F.avg(c).alias(c) for c in df.columns if c in include
    ))
    return df.na.fill(stats.first().asDict())

In [13]:
stazioneDf = fill_with_mean(stazioneDf, ['wind_speed_media'])

In [15]:
stazioneDf.filter(stazioneDf.wind_speed_media.isNull()).count()

0

In [16]:
#n.b per esprimere più condizioni in and o or bisogna racchiudere le condizioni tra parentesi
from pyspark.sql.functions import hour
avgT=stazioneDf.filter((stazioneDf['rad W/mq'].isNotNull())& \
                   ((hour(stazioneDf['data_ora'])<=18)|(hour(stazioneDf['data_ora'])>5)))
avgT=stazioneDf.filter((stazioneDf['rad W/mq'].isNotNull())& \
                   ((hour(stazioneDf['data_ora'])<=18)|(hour(stazioneDf['data_ora'])>5)))
from pyspark.sql.functions import mean
stats=avgT.select([mean('rad W/mq')]).first()
avgRadValue= stats.asDict()
avgRadValue = avgRadValue['avg(rad W/mq)']

In [17]:
avgRadValue

113.87738295555782

In [18]:
from pyspark.sql.functions import when,hour

stazioneDf = stazioneDf.withColumn('rad W/mq',when((stazioneDf['rad W/mq'].isNull())& ((hour(stazioneDf['data_ora'])>18)|\
                                               (hour(stazioneDf['data_ora'])<=5)),0)\
                               .when((stazioneDf['rad W/mq'].isNull())& ((hour(stazioneDf['data_ora'])<=18)|\
                                               (hour(stazioneDf['data_ora'])>5)),avgRadValue)\
                               .otherwise(stazioneDf['rad W/mq']))

In [19]:
stazioneDf.filter((stazioneDf['rad W/mq'].isNull())).count()

0

In [20]:
# con questa opzione della Spark Session impostata a True, non c'è più bisogno di usare show per vedere i dataframe;
# inoltre li posso vedere in modo non sballato
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [21]:
stazioneDf.select(stazioneDf['rad W/mq'],stazioneDf['data_ora'])

rad W/mq,data_ora
75.0,2018-10-12 14:25:15
104.1,2018-10-12 14:30:12
234.4,2018-10-12 14:35:11
247.2,2018-10-12 14:40:13
206.6,2018-10-12 14:45:13
250.3,2018-10-12 14:50:10
307.5,2018-10-12 14:55:13
360.5,2018-10-12 15:00:15
307.5,2018-10-12 15:05:11
388.4,2018-10-12 15:10:13


## Aggiunta della colonna evapotraspirazione

In [22]:
#Et0 = 0.0393 Rs * sqrt(T+9.5)-0.19* Rs^0.6 * lat^0.15 +0.048 *(T+20)(1-UMI/100)* u2^0.7
LATITUDE=0.73

In [23]:
from pyspark.sql.functions import to_date,hour
from pyspark.sql.functions import sum,avg 
# con questa operazione mi calcolo un dataframe in cui ho per ogni giorno la temperatura minima e massima
maxMinTemperaterByDay = stazioneDf.select(to_date(stazioneDf['data_ora']).alias('giorno'),stazioneDf['temp1_min'],stazioneDf['temp1_max'])\
    .groupBy('giorno').agg({"temp1_max":"max","temp1_min":"min"})\
    .withColumnRenamed("max(temp1_max)","t_max").withColumnRenamed("min(temp1_min)","t_min")

In [34]:
#Questa cella serve aggregare i valori per ora, in base da avere valori più realistici di evapotraspirazione
eachHourDf = stazioneDf.select(["data_ora","wind_speed_media","temp1_media","rad W/mq","ur1_media"])\
.groupBy(to_date(stazioneDf['data_ora']).alias("data"),hour(stazioneDf['data_ora'])).agg({"wind_speed_media":"avg","rad W/mq":"avg","ur1_media":"avg"\
                                               ,"temp1_media":"avg"}).\
    withColumnRenamed('avg(rad W/mq)','Rs').withColumnRenamed('avg(temp1_media)','T')\
    .withColumnRenamed('avg(ur1_media)','RH').withColumnRenamed('avg(wind_speed_media)','u2')

In [38]:
# join tra tabella oraria e tabella delle temperature minime e massime, per avere per ogni record della prima
# tabella la temperatura giornaliera massima e minima, utile per il calcolo di Et0
eachHourDf.join(maxMinTemperaterByDay,eachHourDf.data==maxMinTemperaterByDay.giorno,'inner')

data,hour(data_ora),RH,Rs,u2,T,giorno,t_max,t_min
2018-10-17,18,90.60000000000001,0.3,4.27066381798,17.1825,2018-10-17,23.01,12.09
2018-10-18,18,89.125,0.45,5.654218423973333,16.18,2018-10-18,27.01,12.3
2018-10-20,20,78.815,0.5,1.9810546059933332,13.4375,2018-10-20,25.51,8.98
2018-10-29,4,86.75999999999999,0.65,7.67,16.830000000000002,2018-10-29,21.19,9.98
2018-11-01,18,94.44250000000001,0.44999999999999996,0.6025,14.47,2018-11-01,16.07,11.63
2018-11-07,12,79.975,168.425,5.06,15.43,2018-11-07,18.24,5.66
2018-11-08,11,76.39,469.75,2.9975,16.08,2018-11-08,19.08,6.17
2018-11-11,3,96.07,0.95,5.654218423973333,5.8275,2018-11-11,19.62,3.99
2018-11-27,11,82.28,87.1,7.5625,9.447499999999998,2018-11-27,10.89,7.14
2018-12-09,22,75.84,0.9500000000000001,4.8774999999999995,10.955,2018-12-09,15.62,-0.49


In [39]:
eachHourDf = eachHourDf.select('giorno','hour(data_ora)','RH','Rs','u2','T')

data,hour(data_ora),RH,Rs,u2,T
2018-10-17,18,90.60000000000001,0.3,4.27066381798,17.1825
2018-10-18,18,89.125,0.45,5.654218423973333,16.18
2018-10-20,20,78.815,0.5,1.9810546059933332,13.4375
2018-10-29,4,86.75999999999999,0.65,7.67,16.830000000000002
2018-11-01,18,94.44250000000001,0.44999999999999996,0.6025,14.47
2018-11-07,12,79.975,168.425,5.06,15.43
2018-11-08,11,76.39,469.75,2.9975,16.08
2018-11-11,3,96.07,0.95,5.654218423973333,5.8275
2018-11-27,11,82.28,87.1,7.5625,9.447499999999998
2018-12-09,22,75.84,0.9500000000000001,4.8774999999999995,10.955


In [40]:
import math
eachHourDf.withColumn('delta',4098*eachHourDf['T']/((eachHourDf['T']+237.3)**2))

data,hour(data_ora),RH,Rs,u2,T,delta
2018-10-17,18,90.60000000000001,0.3,4.27066381798,17.1825,1.0872826588951532
2018-10-18,18,89.125,0.45,5.654218423973333,16.18,1.0319605003978203
2018-10-20,20,78.815,0.5,1.9810546059933332,13.4375,0.8758945993908427
2018-10-29,4,86.75999999999999,0.65,7.67,16.830000000000002,1.0679334680651833
2018-11-01,18,94.44250000000001,0.44999999999999996,0.6025,14.47,0.9354757316096938
2018-11-07,12,79.975,168.425,5.06,15.43,0.9899751321433905
2018-11-08,11,76.39,469.75,2.9975,16.08,1.0263921807940448
2018-11-11,3,96.07,0.95,5.654218423973333,5.8275,0.40400437435599107
2018-11-27,11,82.28,87.1,7.5625,9.447499999999998,0.6358919576004304
2018-12-09,22,75.84,0.9500000000000001,4.8774999999999995,10.955,0.7284308451562326


## Stima della Et0 con la formula di Valiantzas

In [41]:
#La radiazione solare viene convertita in MJ/day che è l'unita di misura richiesta per la formula di Valiantzas
eachHourDf = eachHourDf.withColumn('Rs',eachHourDf['Rs']*0.0864)

In [42]:
eachHourDf = eachHourDf.withColumn('Et0',0.0393*eachHourDf['Rs']*((eachHourDf['T']+9.5)**0.5)-0.19*(eachHourDf['Rs']**0.6)\
                                   *(0.73**0.15)+0.048*(eachHourDf['T']+20)\
                                   *(1-(eachHourDf['RH']/100))*(eachHourDf['U2']**0.7))

In [43]:
eachHourDf.select('data','Et0').count()

5823

In [46]:
from pyspark.ml.feature import VectorAssembler

# prendiamo le temperature medie, minime emassime come feature

vectorAssembler = VectorAssembler(inputCols = ['RH','Rs','T','u2'], outputCol = 'features')
vstazione_df = vectorAssembler.transform(eachHourDf)
vstazione_df = vstazione_df.select(['features', 'Et0'])
vstazione_df

features,Et0
[90.6000000000000...,0.44851653104635747
"[89.125,0.0388800...",0.616953535945681
"[78.815,0.0432,13...",0.5293128453593872
[86.7599999999999...,0.9534100609984606
[94.4425000000000...,0.04614893374323514
"[79.975,14.551920...",3.0112619427221468
"[76.39,40.5864000...",7.276769233111957
"[96.07,0.08208,5....",0.13601424532850462
"[82.28,7.52544,9....",1.7113178838919438
"[75.84,0.08208000...",1.0625971517701907


In [47]:
# divisione training set e test set
splits = vstazione_df.randomSplit([0.8,0.2])
train_df = splits[0]
test_df = splits[1]

In [48]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='Et0', maxIter=20, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.0468760680033233,0.15365411094286635,0.03157557041568009,0.07971281711748317]
Intercept: 3.993764776030423


In [102]:
# calcolo del root means square. Siccome stiamo lavorando con dati della radiazione di ordini di grandezza
# molto alti, riteniamo accettabile un errore di circa 150

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.706690
r2: 0.958735


In [49]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 0.726763


In [50]:
#il modello viene salvato automaticamente su hdfs essendo configurato spark su yarn
lr_model.save("output/linear_model_evapotranspiration")

## Uso di random forest per il modello di regressione

In [53]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(featuresCol="features",labelCol="Et0")
rf_model = rf.fit(train_df)
predictions = rf_model.transform(test_df)

In [52]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="Et0", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.607774


In [54]:
rf_model.save("output/random_forest_model_evapotranspiration")

## Uso di gradient boost regression 

In [57]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(train_df)
gbt = GBTRegressor(featuresCol="features",labelCol = 'Et0', maxIter=10)
pipeline = Pipeline(stages=[featureIndexer, gbt])
gbt_model = pipeline.fit(train_df)

In [58]:
predictions = gbt_model.transform(test_df)

In [60]:
evaluator = RegressionEvaluator(
    labelCol="Et0", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = gbt_model.stages[1]
print(gbtModel)  # summary only

Root Mean Squared Error (RMSE) on test data = 0.606528
GBTRegressionModel (uid=GBTRegressor_b9a8f013af8c) with 10 trees


In [61]:
gbt_model.save('output/gbt_model_evapotranspiration')